In [ ]:
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install nltk

In [6]:
import pandas as pd

In [754]:
import os
naming_convention = "ML_Dataset_email_classifier_"
folder_path = "datasets"

# Get the list of all files in the folder
all_files = os.listdir(folder_path)
dataset_files = [os.path.join(folder_path, file) for file in all_files]
dataset_files.sort(key=os.path.getctime, reverse=True)

In [ ]:
# # initialize datasets here
# dataset = "Internship Tracker - ML_Dataset_email_classifier.csv"
# dataset_kavya = "ML_Dataset_email_classifier_kavya.csv"
# dataset_hamza = "ML_Dataset_email_classifier_hamza.csv"
# dataset_ary = "ML_Dataset_email_classifier_new_ary.csv"
# dataset_sankaet = "ML_Dataset_email_classifier_sankaet.csv"

In [870]:
def read_datasets(dataset_files):
    datasets = {}
    for file in dataset_files:
        name = "dataset_" + file.split("/")[-1].split(".")[0].split("_")[-1]
        datasets[name] = pd.read_csv(file)
    return datasets
datasets = read_datasets(dataset_files)
print(datasets.keys())

added_datasets = set()

dict_keys(['dataset_yoomin', 'dataset_muskan', 'dataset_sankaet', 'dataset_original', 'dataset_hamza', 'dataset_ary', 'dataset_kavya'])


In [871]:
# Load the dataset
# create empty dataframe
# data = pd.DataFrame() # to reset the data
for index, (name, dataset) in enumerate(datasets.items()):
    if name == "dataset_original" or index == 0:
        #skip over the original dataset
        continue
    elif name not in added_datasets:
        data = pd.concat([data, datasets[name]], ignore_index=True)
        added_datasets.add(name)

print(added_datasets)

{'dataset_muskan', 'dataset_sankaet', 'dataset_ary', 'dataset_hamza', 'dataset_kavya'}


In [872]:
# Check the first few rows of your dataset
data.head()
data.info()
data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3719 entries, 0 to 3718
Data columns (total 5 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   email subject                            3719 non-null   object 
 1   email body                               3718 non-null   object 
 2   binary classification (expected output)  1843 non-null   float64
 3   cleaned email subject                    905 non-null    object 
 4   cleaned email body                       905 non-null    object 
dtypes: float64(1), object(4)
memory usage: 145.4+ KB


,binary classification (expected output)
count,1843.000000
mean,0.684211
std,0.464956
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [ ]:
pd.set_option('display.max_rows', None)
for index, row in data.iterrows():
    print(str(index) + " " + f"Email Subject: {row['email subject']} - Classification: {row['binary classification (expected output)']}")
pd.reset_option('display.max_rows')

In [931]:
import pandas as pd

# Assuming 'datasets' is a dictionary of DataFrames and 'first_dataset_name' is the key for the first dataset
first_dataset_name = list(datasets.keys())[0]
print(first_dataset_name)
first_dataset = datasets[first_dataset_name]

for index, row in data.iterrows():
    mask = (first_dataset['email subject'] == row['email subject']) & (first_dataset['email body'] == row['email body'])
    first_dataset.loc[mask, 'binary classification (expected output)'] = row['binary classification (expected output)']

first_dataset.info()


dataset_yoomin
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87 entries, 0 to 86
Data columns (total 3 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   email subject                            87 non-null     object 
 1   email body                               87 non-null     object 
 2   binary classification (expected output)  80 non-null     float64
dtypes: float64(1), object(2)
memory usage: 2.2+ KB


In [932]:
# Function to sample from unlabeled data without duplicates
def sample_unlabeled_data(dataset, n, sampled_indices):
    # Filter the DataFrame to include only the unlabeled rows
    unlabeled_data = dataset[dataset['binary classification (expected output)'].isnull()]
    print(f"Total unlabeled data: {len(unlabeled_data)}")
    # Get available indices that have not been sampled yet
    available_indices = unlabeled_data.index.difference(sampled_indices)
    print(f"Available indices: {len(available_indices)}")
    # Check if there are enough available indices to sample
    if len(available_indices) < n:
        print(f"Not enough available indices to sample {n} rows. Sampling {len(available_indices)} rows instead.")
        n = len(available_indices)
    # Sample from the available indices
    sampled_data = unlabeled_data.loc[available_indices].sample(n=n, random_state=42)
    # Update the set of sampled indices
    sampled_indices.update(sampled_data.index)
    return sampled_data

sampled_indices = set()
sample = sample_unlabeled_data(first_dataset, 20, sampled_indices)

# Sample 100 rows from the unlabeled data in the first dataset
while len(sampled_indices) < len(data[data['binary classification (expected output)'].isnull()]):
    sample = sample_unlabeled_data(data, 20, sampled_indices)


Total unlabeled data: 7
Available indices: 7
Not enough available indices to sample 20 rows. Sampling 7 rows instead.


In [933]:
data = pd.concat([data, sample], ignore_index=True)
print(data)

                                          email subject  \
0                  Thank you for applying to Salesforce   
1     [Action Requested by 1/10]: Your Wayfair Softw...   
2     Your Application for Software Engineering Inte...   
3     Update on your application: We're moving forwa...   
4                        BlackRock | Application update   
...                                                 ...   
1885  Your application for Product Manager II, Curat...   
1886  Following up on your recent application to Google   
1887  Your application for Staff Product Manager, Tr...   
1888  Thank you for applying to Salesforce / Associa...   
1889  You have applied to Cisco Job Product Manageme...   

                                             email body  \
0     Hello, Thank you for considering a career with...   
1     Hi there, Thank you for your interest in the S...   
2     Dear Hamza, Thank you again for taking the tim...   
3     Dear Hamza , Thank you again for taking the ti...

In [934]:
data.head()
data.info()
data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1890 entries, 0 to 1889
Data columns (total 5 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   email subject                            1890 non-null   object 
 1   email body                               1890 non-null   object 
 2   binary classification (expected output)  1883 non-null   float64
 3   cleaned email subject                    905 non-null    object 
 4   cleaned email body                       905 non-null    object 
dtypes: float64(1), object(4)
memory usage: 74.0+ KB


,binary classification (expected output)
count,1883.000000
mean,0.689326
std,0.462893
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [935]:
# Check for missing values
data.isnull().sum()

# Drop rows with missing values (if necessary)
data.dropna(subset=['email subject', 'email body'], inplace=True)

#drop rows that are duplicates
data.drop_duplicates(subset=['email subject', 'email body'], inplace=True)

# Optionally, inspect cleaned data
data.head()
data.info()
data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1890 entries, 0 to 1889
Data columns (total 5 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   email subject                            1890 non-null   object 
 1   email body                               1890 non-null   object 
 2   binary classification (expected output)  1883 non-null   float64
 3   cleaned email subject                    905 non-null    object 
 4   cleaned email body                       905 non-null    object 
dtypes: float64(1), object(4)
memory usage: 74.0+ KB


,binary classification (expected output)
count,1883.000000
mean,0.689326
std,0.462893
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [936]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

# Define a function to clean the text
def clean_text(text):
    # Tokenize and lower case the text
    text = text.lower()
    tokens = word_tokenize(text)
    
    # Remove stopwords and punctuation
    stop_words = set(stopwords.words('english'))
    cleaned_tokens = [word for word in tokens if word not in stop_words and word not in string.punctuation]
    
    return " ".join(cleaned_tokens)

clean_data = data.copy()

# Apply the cleaning function to both 'email subject' and 'email body'
clean_data['cleaned email subject'] = data['email subject'].apply(clean_text)
clean_data['cleaned email body'] = data['email body'].apply(clean_text)


In [849]:
import nltk

print(nltk.__version__)


3.8.1


In [937]:
import os
def save_accuracy_score(accuracy, file_path='accuracy_scores.csv'):
    # Check if the file exists and is not empty
    if not os.path.exists(file_path) or os.path.getsize(file_path) == 0:
        # Create a new DataFrame with headers if the file does not exist or is empty
        scores_df = pd.DataFrame(columns=['Iteration', 'Accuracy'])
    else:
        # Load existing scores if the file exists and is not empty
        scores_df = pd.read_csv(file_path)

    # Append the new accuracy score
    new_score = {'Iteration': len(scores_df) + 1, 'Accuracy': accuracy}
    scores_df = pd.concat([scores_df, pd.DataFrame([new_score])], ignore_index=True)

    # Save the updated scores to the file
    scores_df.to_csv(file_path, index=False)

In [938]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [939]:
# Combine the subject and body into a single text feature
clean_data['combined_text'] = clean_data['cleaned email subject'] + " " + clean_data['cleaned email body']

# Define your features (X) and labels (y)
X = clean_data['combined_text']
y = data['binary classification (expected output)']  # This should be a column with binary labels (1 for job applications, 0 otherwise)

In [940]:
# Encode labels into binary format
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [941]:
from sklearn.linear_model import LogisticRegression

# Align indices before separating labeled and unlabeled data
aligned_index = clean_data.index.intersection(data.index)

# Separate labeled and unlabeled data
labeled_data = clean_data.loc[aligned_index][data['binary classification (expected output)'].notnull()]
before_labeled_data = len(labeled_data)

unlabeled_data = clean_data.loc[aligned_index][data['binary classification (expected output)'].isnull()]
before_unlabeled_data = len(unlabeled_data)

print("Number of labeled data points:", before_labeled_data)
print("Number of unlabeled data points:", before_unlabeled_data)
# check whether unlabeled data is empty
if unlabeled_data.empty:
    print("All the data is labeled")

Number of labeled data points: 1883
Number of unlabeled data points: 7


In [942]:
def correct_prediction(incorrect_index, run_function):
    if incorrect_index is None or not run_function:
        return
    try:
        # Correct the prediction for the given index
        unlabeled_data.at[incorrect_index, 'predictions'] = 1
        # print("This works")
    except:
        print("Index out of range")
        return

In [943]:
print(unlabeled_data)

                                          email subject  \
1883                  Thank you for applying to Samsara   
1884        Barclays: We’ve received your application!    
1885  Your application for Product Manager II, Curat...   
1886  Following up on your recent application to Google   
1887  Your application for Staff Product Manager, Tr...   
1888  Thank you for applying to Salesforce / Associa...   
1889  You have applied to Cisco Job Product Manageme...   

                                             email body  \
1883  Dear Yoomin, Thank you for applying to the Ass...   
1884  Subject: Yoomin Choi ‎- 90392884 Full Stack De...   
1885  Hi Yoomin, Thank you for your interest in join...   
1886  Thanks so much for applying with us Hi Yoomin,...   
1887  Hi Yoomin, Thank you for your interest in join...   
1888  Dear Yoomin, Thank you for considering a caree...   
1889  Hi Yoomin, Thank you for applying at Cisco! We...   

      binary classification (expected output)  \
1883 

In [944]:
import joblib

tfidf_vectorizer = joblib.load('tfidf_vectorizer.pkl')
classifier = joblib.load('email_classifier.pkl')

predictions = classifier.predict(tfidf_vectorizer.transform(unlabeled_data['combined_text']))
print(len(predictions))
print(predictions)
unlabeled_data['predictions'] = predictions

7
[1 1 1 1 1 1 1]


In [945]:
incorrect_index_list = []
for i in incorrect_index_list:
    correct_prediction(i, False)

unlabeled_data.to_csv("new_sample_set_with_predictions.csv", index=False)

# print(unlabeled_data['combined_text'], unlabeled_data['predictions']) 
# print(unlabeled_data.head())
i = 0
for text, prediction in zip(unlabeled_data['email body'], unlabeled_data['predictions']):
    print(f"Index: {unlabeled_data.index[i]} Text: {text}\nPrediction: {prediction}\n")
    i+=1

Index: 1883 Text: Dear Yoomin, Thank you for applying to the Associate Product Manager role and for choosing Samsara in your job search! Your interest means a lot to us, and we're grateful for your application. Our recruiters are actively reviewing applications every day, and we will be in touch about next steps if you are selected to move forward in the application process. In the meantime, please get to know us more: Learn about our mission and values Follow us on LinkedIn for the latest updates Explore Samsara Careers for additional opportunities Warm regards, Samsara’s Recruiting Team ** Please note: Do not reply directly to this email.
Prediction: 1

Index: 1884 Text: Subject: Yoomin Choi ‎- 90392884 Full Stack Developer Barclays Email Classification: Restricted - External Dear Yoomin, Thank you for applying to Barclays and for the time and effort you’ve put into exploring a role with us. We wanted to let you know we’ve received your application and are currently reviewing it. You

In [895]:
unlabeled_data.at[1859, 'predictions'] = 0

In [946]:
def mapPredictionsToIndices():
    index_prediction_map = {}
    for i, pred in enumerate(unlabeled_data['predictions']):
        if pred not in index_prediction_map:
            index_prediction_map[pred] = []
        index_prediction_map[pred].append(unlabeled_data.index[i])
    
    return index_prediction_map

In [947]:
index_prediction_map = mapPredictionsToIndices()

for prediction, indices in index_prediction_map.items():
    print(f"Prediction: {prediction}, Indices: {indices}")


Prediction: 1, Indices: [1883, 1884, 1885, 1886, 1887, 1888, 1889]


In [948]:
# Predict probabilities on unlabeled data
X_unlabeled = unlabeled_data['combined_text']
try:
    print("Unlabeled data is not empty")
    X_unlabeled_tfidf = tfidf_vectorizer.transform(X_unlabeled)
except:
    print("Unlabeled data is empty")

Unlabeled data is not empty


In [949]:
probabilities = classifier.predict_proba(X_unlabeled_tfidf)
uncertainty = 1 - probabilities.max(axis=1)  # Higher uncertainty = lower confidence
unlabeled_data['uncertainty'] = uncertainty

# Sort by uncertainty
most_uncertain = unlabeled_data.sort_values(by='uncertainty', ascending=False)
uncertain_num = len(unlabeled_data)
most_uncertain_subset = most_uncertain.index[:uncertain_num]
most_uncertain[['email subject', 'email body', 'uncertainty']].index[:uncertain_num]
# print the email body of the uncertain_num most uncertain emails
for i in range(uncertain_num):
    print(most_uncertain.index[i], most_uncertain['uncertainty'].iloc[i], most_uncertain['email body'].iloc[i])


1886 0.20671578322847606 Thanks so much for applying with us Hi Yoomin, A Googler recently invited you to apply for the UX Engineer, Early Career - Sunnyvale role. We carefully reviewed your background and experience, and decided not to proceed with your application at this time. Although this role didn't work out, we may contact you if we come across another opening that we think could interest you and that matches your skills and experience. Also, if you applied for any other roles with us recently, look for an update on them soon. Thanks again for your interest, Ayesha Jabeen (xWF), Google recruiting © 2018 Google LLC 1600 Amphitheatre Parkway, Mountain View, CA 94043 You have received this message because you recently applied for a job at Google.
1889 0.14377319593317595 Hi Yoomin, Thank you for applying at Cisco! We think it's a great place to work, and we're excited that you're interested in joining us as a Product Management Specialist I (Full Time) United States, Job ID 1426742

In [950]:
indices = []
indices = index_prediction_map[1]
# I want to label a set number of emails as job applications which ill do with the index
# of the most uncertain emails
# get the index of the uncertain_num most uncertain emails
# index = most_uncertain.index[:uncertain_num]
# print(index)
# label the emails as job applications
data.loc[indices, 'binary classification (expected output)'] = 1.0
# check if the emails have been labeled
print(data.loc[indices, 'binary classification (expected output)'])

1883    1.0
1884    1.0
1885    1.0
1886    1.0
1887    1.0
1888    1.0
1889    1.0
Name: binary classification (expected output), dtype: float64


In [951]:
import numpy as np
# label the other indices in the subset as not job applications
not_indices = [val for val in most_uncertain_subset if val not in indices]
data.loc[not_indices, 'binary classification (expected output)'] = 0.0
print(data.loc[not_indices, 'binary classification (expected output)'])

Series([], Name: binary classification (expected output), dtype: float64)


In [952]:
unlabeled_data = clean_data.loc[aligned_index][data['binary classification (expected output)'].isnull()]
after_unlabeled_data = len(unlabeled_data)
print(after_unlabeled_data)
labeled_data = clean_data.loc[aligned_index][data['binary classification (expected output)'].notnull()]
after_labeled_data = len(labeled_data)
print(after_labeled_data)

print(after_labeled_data - before_labeled_data)
print(after_unlabeled_data - before_unlabeled_data)

0
1890
7
-7


In [744]:
data.at[438, 'binary classification (expected output)'] = 0.0
# show results
print(data.loc[1708, 'binary classification (expected output)'])

0.0


In [953]:
# Define your features (X) and labels (y)
X = clean_data['combined_text']
y = data['binary classification (expected output)']  # This should be a column with binary labels (1 for job applications, 0 otherwise)

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Initialize the TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words='english', ngram_range=(1, 2))

# Fit and transform the training data, transform the test data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)


In [954]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Initialize and train the classifier
classifier = LogisticRegression()
classifier.fit(X_train_tfidf, y_train)

# Make predictions
y_pred = classifier.predict(X_test_tfidf)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report(y_test, y_pred))

save_accuracy_score(accuracy)


Accuracy: 0.955026455026455
Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.88      0.93       126
           1       0.94      0.99      0.97       252

    accuracy                           0.96       378
   macro avg       0.96      0.94      0.95       378
weighted avg       0.96      0.96      0.95       378



In [955]:
import joblib

# Save the vectorizer and model
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.pkl')
joblib.dump(classifier, 'email_classifier.pkl')

joblib.dump(tfidf_vectorizer, 'emailClassifier/tfidf_vectorizer.pkl')
joblib.dump(classifier, 'emailClassifier/email_classifier.pkl')


['emailClassifier/email_classifier.pkl']

In [752]:
print(len(tfidf_vectorizer.vocabulary_))

5000


In [692]:
# Load vectorizer and model
loaded_vectorizer = joblib.load('tfidf_vectorizer.pkl')
loaded_model = joblib.load('email_classifier.pkl')

# Preprocess and vectorize the new email
new_email = "This is a job application for the software engineer position."
new_email_tfidf = loaded_vectorizer.transform([new_email])

# Predict
prediction = loaded_model.predict(new_email_tfidf)
print("Prediction:", "Job Application" if prediction[0] == 1 else "Not a Job Application")

Prediction: Job Application


In [ ]:
!python --version
!pip show scikit-learn joblib
# code to get the numpy version?
import numpy as np
print(np.__version__)


Python 3.11.5
Name: scikit-learn
Version: 1.2.2
Summary: A set of python modules for machine learning and data mining
Home-page: http://scikit-learn.org
Author: 
Author-email: 
License: new BSD
Location: /Users/ary/anaconda3/lib/python3.11/site-packages
Requires: joblib, numpy, scipy, threadpoolctl
Required-by: imbalanced-learn
---
Name: joblib
Version: 1.2.0
Summary: Lightweight pipelining with Python functions
Home-page: https://joblib.readthedocs.io
Author: Gael Varoquaux
Author-email: gael.varoquaux@normalesup.org
License: BSD
Location: /Users/ary/anaconda3/lib/python3.11/site-packages
Requires: 
Required-by: imbalanced-learn, nltk, scikit-learn
1.26.3
